In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split

pre_trained = True

In [2]:

def distance_canberra_mat(d1, d2):
    ### Don't seems to work in scipy    
    divide = abs(d1) + abs(d2)    
    distance = np.sum(np.divide(abs(d1-d2), divide, where=divide!=0), axis=1)    
    return distance
    
def distance_cosine_mat(d1, d2):
    ### Don't seems to work in scipy
    val = np.sum(d1*d2, axis=1)
    divide = np.sqrt(np.sum((d1*d1), axis=1))*np.sqrt(np.sum((d2*d2), axis=1))
    return 1 - np.divide(val, divide)

def distance_khi2_mat_w(d1, d2, w):
    divide = (d1 + d2)*w    
    distance = np.sum(np.divide((d1-d2)**2, divide, where=divide!=0), axis=1)   
    return distance

def distance_khi2_mat(d1, d2):
    divide = d1 + d2    
    distance = np.sum(np.divide((d1-d2)**2, divide, where=divide!=0), axis=1)   
    return distance


def distance_braycurtis_mat(d1, d2):
    divide = np.sum(abs(d1 + d2), axis=1)
    distance = np.divide(np.sum(abs(d1-d2), axis=1), divide)   
    return distance


def distance_eucl_mat(d1, d2):
    distance = np.sqrt(np.sum((d1-d2)**2, axis=1))   
    return distance

In [3]:
train_label = pd.read_csv("data/train_with_ids.csv")

In [4]:
#train_label[["QS","QR"]].to_csv("data/train_labels.csv")

In [4]:
train_data = None


if(pre_trained==False) :
    xtrain=pd.read_csv("data/xtrain_challenge.csv")
    train_data = xtrain
    
    cols_qs = list(xtrain.columns)[:13]
    cols_qr = list(xtrain.columns)[13:26]
    cols_S1_11 = list(xtrain.columns)[26:37]
    


    # On créé des features artificielles:
    # Q1-13 Q13-26 sont des identifiants uniques d'un visage
    # On suppose que Q1-13 et Q13-26 pourrait etre inversé
    # On regarde si on n'aurait pas des liaisons à faire en plus

    # Merge des colonnes

    train_data["QS"] = train_data[cols_qs].apply(
        lambda x: '_'.join(x.astype(str)),
        axis=1
    )

    # Merge des colonnes

    train_data["QR"] = train_data[cols_qr].apply(
        lambda x: '_'.join(x.astype(str)),
        axis=1
    )
    train_data[['QS','QR']].to_csv("data/train_labels.csv")
else:
    train_data = pd.read_csv("data/xtrain_challenge.csv")
    
  
    
cols_qs = list(train_data.columns)[:13]
cols_qr = list(train_data.columns)[13:26]
cols_S1_11 = list(train_data.columns)[26:37]
    

In [5]:
train_data["canberra"]=distance_canberra_mat(train_data[cols_qr].values, train_data[cols_qs].values)
train_data["cosine"]=distance_cosine_mat(train_data[cols_qr].values, train_data[cols_qs].values)
train_data["khi2"]=distance_khi2_mat(train_data[cols_qr].values,train_data[cols_qs].values)
train_data["braycurtis"]=distance_braycurtis_mat(train_data[cols_qr].values,train_data[cols_qs].values)
train_data["euclidian"]=distance_eucl_mat(train_data[cols_qr].values,train_data[cols_qs].values)
ytrain=pd.read_csv("data/ytrain_challenge.csv")
xtrain = train_data

In [6]:
train_data['QS'] = train_label['QS'].apply(lambda x: hash(x))

In [7]:
train_data['QR'] = train_label['QR'].apply(lambda x: hash(x))

In [ ]:
train_data.head(10)


In [ ]:
#train_data[train_data.label==1].groupby("QS")

In [ ]:
for c in list(xtrain.columns):    
    xtrain[c].plot(kind='hist', bins=30)
    plt.title(c)
    plt.show()


In [10]:
train_data.describe()

,qs1,qs2,qs3,qs4,qs5,qs6,qs7,qs8,qs9,qs10,...,s9,s10,s11,canberra,cosine,khi2,braycurtis,euclidian,QS,QR
count,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,...,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06,9.800713e+06
mean,9.918364e-01,5.252516e-03,2.858537e-03,-2.243750e-01,-5.766788e+00,-7.422710e-01,1.264598e+00,9.991708e-01,8.592701e-01,2.561005e+02,...,2.740128e+03,3.306104e+03,2.717576e+03,4.243066e+00,1.659784e-02,1.070081e+01,1.727854e-01,7.434250e+01,4.732326e+16,2.644986e+12
std,6.377177e-02,5.143305e-02,3.607721e-02,9.952588e+00,8.868902e+00,4.565691e+00,1.158636e+00,8.948037e-01,7.733346e-01,7.512713e+01,...,1.361459e+03,8.474081e+02,1.207085e+03,1.092331e+00,9.567894e-02,1.115858e+03,1.243161e-01,5.313885e+01,5.337736e+18,5.324747e+18
min,0.000000e+00,0.000000e+00,0.000000e+00,-7.820000e+01,-6.771000e+01,-3.482000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.980000e+02,...,1.238680e+03,2.160870e+03,1.123050e+03,1.353562e-02,4.905670e-07,-3.051626e+05,1.599573e-03,7.253964e-01,-9.223315e+18,-9.223370e+18
25%,1.000000e+00,0.000000e+00,0.000000e+00,-3.220000e+00,-1.263000e+01,-2.850000e+00,4.700000e-01,3.500000e-01,3.100000e-01,2.112600e+02,...,2.162600e+03,2.926560e+03,2.170650e+03,3.491038e+00,1.347677e-03,-1.366981e+01,9.077034e-02,3.476738e+01,-4.609988e+18,-4.609368e+18
50%,1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e-02,-4.870000e+00,-6.000000e-01,1.000000e+00,7.600000e-01,6.700000e-01,2.641500e+02,...,2.367710e+03,3.072890e+03,2.386120e+03,4.184153e+00,3.365515e-03,6.826790e+00,1.425700e-01,6.041731e+01,7.293860e+16,1.416852e+15
75%,1.000000e+00,0.000000e+00,0.000000e+00,3.140000e+00,9.600000e-01,1.430000e+00,1.740000e+00,1.380000e+00,1.190000e+00,3.091400e+02,...,2.659910e+03,3.283640e+03,2.698510e+03,4.927892e+00,8.113502e-03,3.371280e+01,2.194023e-01,1.005729e+02,4.664395e+18,4.612868e+18
max,1.000000e+00,1.000000e+00,1.000000e+00,5.934000e+01,2.668000e+01,6.482000e+01,3.873000e+01,1.568000e+01,2.442000e+01,4.704700e+02,...,1.411044e+04,1.013524e+04,1.187424e+04,1.184035e+01,1.981039e+00,5.611421e+05,4.631523e+00,1.947190e+03,9.223347e+18,9.223372e+18


In [9]:
# Load test data
xtest = pd.read_csv('data/xtest_challenge.csv')

In [10]:
xtest["canberra"]=distance_canberra_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["cosine"]=distance_cosine_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["khi2"]=distance_khi2_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["braycurtis"]=distance_braycurtis_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["euclidian"]=distance_eucl_mat(xtest[cols_qr].values,xtest[cols_qs].values)

In [13]:
xtest.describe()

,qs1,qs2,qs3,qs4,qs5,qs6,qs7,qs8,qs9,qs10,...,s7,s8,s9,s10,s11,canberra,cosine,khi2,braycurtis,euclidian
count,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,...,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06,3.768311e+06
mean,9.927863e-01,4.310032e-03,2.860953e-03,1.574280e-01,-5.835045e+00,-5.682256e-01,1.242306e+00,9.726340e-01,8.281594e-01,2.518609e+02,...,2.198468e+03,2.285049e+03,2.567436e+03,3.198011e+03,2.570077e+03,4.235104e+00,1.638737e-02,1.267799e+01,1.721996e-01,7.323465e+01
std,5.887766e-02,4.599935e-02,3.586661e-02,9.244392e+00,8.968709e+00,4.316559e+00,1.167329e+00,8.767055e-01,7.400339e-01,7.495040e+01,...,1.061593e+03,1.085586e+03,9.946063e+02,6.084623e+02,8.879480e+02,1.091056e+00,9.384112e-02,1.045572e+03,1.257299e-01,5.256339e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-5.761000e+01,-5.143000e+01,-2.531000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.651600e+02,...,3.487100e+02,4.273800e+02,1.286340e+03,2.236650e+03,1.134200e+03,2.263297e-03,1.658806e-07,-2.444028e+05,2.303882e-04,1.200000e-01
25%,1.000000e+00,0.000000e+00,0.000000e+00,-2.640000e+00,-1.280000e+01,-2.510000e+00,4.500000e-01,3.400000e-01,3.000000e-01,2.071100e+02,...,1.717600e+03,1.793770e+03,2.157940e+03,2.923820e+03,2.169140e+03,3.483525e+00,1.262143e-03,-1.186449e+01,8.948902e-02,3.391126e+01
50%,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e-01,-5.090000e+00,-5.200000e-01,9.600000e-01,7.400000e-01,6.500000e-01,2.582500e+02,...,1.975820e+03,2.060690e+03,2.358030e+03,3.066190e+03,2.379410e+03,4.176517e+00,3.188195e-03,7.165059e+00,1.417600e-01,5.954016e+01
75%,1.000000e+00,0.000000e+00,0.000000e+00,2.990000e+00,9.000000e-01,1.380000e+00,1.710000e+00,1.350000e+00,1.140000e+00,3.050300e+02,...,2.321820e+03,2.414130e+03,2.625150e+03,3.259740e+03,2.664690e+03,4.918782e+00,7.812440e-03,3.324913e+01,2.191031e-01,9.941349e+01
max,1.000000e+00,1.000000e+00,1.000000e+00,6.297000e+01,2.413000e+01,2.770000e+01,3.340000e+01,9.430000e+00,1.420000e+01,4.814200e+02,...,1.306624e+04,1.471272e+04,1.373343e+04,9.316200e+03,1.148053e+04,1.182820e+01,1.984838e+00,3.916112e+05,6.203517e+00,1.485462e+03


On va calculer des distances entre les features Q1-13 et Q13-26

https://www.cs.toronto.edu/~ranzato/publications/taigman_cvpr14.pdf


In [14]:
# Calcul d'un SVM pour connaitre les coefficients des features

In [11]:
from sklearn import svm
clf = svm.LinearSVC()

d1 = train_data[cols_qs].values
d2 = train_data[cols_qr].values

divide = d1 + d2 
X = np.divide((d1-d2)**2, divide, where=divide!=0)

X_SVM, X_SVM_test, y_SVM, y_SVM_test = train_test_split(X, ytrain, test_size=0.98) # SVM on 2% of data

X_SVM.shape

(196014, 13)

In [12]:
clf.fit(X_SVM, y_SVM)

C:\Users\martinez\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\martinez\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [13]:
w = clf.coef_

In [15]:
xtest["khi2 W"]=distance_khi2_mat_w(xtest[cols_qr].values,xtest[cols_qs].values, w)
train_data["khi2_W"]=distance_khi2_mat_w(train_data[cols_qr].values,train_data[cols_qs].values,w)

In [ ]:
ytrain[xtrain.qr9 >xtrain.qs9.max()].index

In [ ]:
ytrain[xtrain.qr1 >xtrain.qs1.max()].index

In [ ]:
xtest[xtest.qr1>1].index

In [ ]:
xtest[xtest.qr9> xtrain.qs9.max()].index

In [16]:
xtrain_clean = xtrain[xtrain.qr1<=1]



In [ ]:
xtrain_clean.describe()

In [44]:
qs_unique = len(xtrain_clean.QS.unique())

##### for c in list(xtrain_clean.columns):
    xtrain_clean[c].plot(kind='hist', bins=30, logy=True)   
    plt.title(c)
    plt.show()    
    xtrain_clean[c].plot(kind='hist', bins=30)
    plt.title(c)
    plt.show()

In [45]:
len(xtrain_clean.QR.unique())

9145789

In [54]:
ytrain_clean = ytrain[xtrain.qr1 <=1]
yone = ytrain_clean[ytrain_clean==1]

import math

choosen = sk.utils.random.sample_without_replacement(qs_unique,qs_unique//8)

# get the hash of the data
qs = xtrain_clean.QS.iloc[choosen].values





In [55]:
indices = np.in1d(xtrain_clean.QS.values, qs)

In [56]:
xtrain_clean.QS[indices].count()

2508140

In [57]:

#X_train, X_test, y_train, y_test = train_test_split(xtrain_clean, ytrain[xtrain.qr1 <=1], test_size=0.33, random_state=42)

X_train= xtrain_clean[][~indices]
X_test=xtrain_clean[indices]
y_train= ytrain_clean[~indices]
y_test=ytrain_clean[indices]


In [59]:
cols_active = list(train_data.columns)

remove_list=["QS","QR"] #, "qs1","qr1","qs2","qr2","qs3","qr3","qs12","qr12"]    
for i in remove_list: 
    try: 
        cols_active.remove(i) 
    except ValueError: 
        pass
    

In [62]:

import lightgbm 
# making lgbm datasets for train and valid
d_train = lightgbm.Dataset(X_train[cols_active], y_train)
d_valid = lightgbm.Dataset(X_test[cols_active], y_test)
    
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_error',
    'is_unbalance': 'false',
    'boosting': 'gbdt',
    'num_leaves': 100,
    'max_bin':1024,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.2,
    'bagging_freq': 5,
    'learning_rate': 0.01,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       d_train,
                       valid_sets=d_valid,
                       num_boost_round=1000,
                       early_stopping_rounds=50)   
    # making prediciton for one column


MemoryError: Unable to allocate 2.34 GiB for an array with shape (43, 7292505) and data type float64

In [ ]:

plt.rcParams['figure.figsize'] = [20, 10]
lightgbm.plot_importance(model)

In [38]:
#model.save_model('lgbm_distance.txt')

soft = model.predict(X_test)

print('Fininshed Training')
soft

Fininshed Training


array([0.03252096, 0.03252096, 0.03252096, ..., 0.03252096, 0.04047605,
       0.03252096])

In [39]:

yvalid = soft

In [40]:
N = np.sum(y_test == 0).values
P = np.sum(y_test == 1).values



In [41]:

yvalid_scoreordered = y_test.values[np.argsort(yvalid)]


FP = 0.0
TP = 0.0
val = 0
print(N)
print(P)
for i in range(len(y_test) - 1, -1, -1):        
    if (yvalid_scoreordered[i] == 1):
        TP = TP + 1
    else:
        FP = FP + 1
        #print(X_test[cols].iloc(i))
    if (FP / N > 10**-4):
        FP = FP - 1
        break
print("For the smallest FPR <= 10^-4 (i.e., ", FP / N, ") TPR = ", TP / P, ".", sep = "")

[3012122]
[222091]
For the smallest FPR <= 10^-4 (i.e., [9.99295513e-05]) TPR = [0.87299801].


In [33]:
# Load test data
xtest = pd.read_csv('data/xtest_challenge.csv')
xtest["canberra"]=distance_canberra_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["cosine"]=distance_cosine_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["khi2"]=distance_khi2_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["braycurtis"]=distance_braycurtis_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["euclidian"]=distance_eucl_mat(xtest[cols_qr].values,xtest[cols_qs].values)

In [35]:
# Classify the provided test data
ytest = model.predict(xtest.values)
print(ytest.shape)
np.savetxt('ytest_lbgm_distance_s1.csv', ytest, fmt = '%1.15f', delimiter=',')

(3768311,)
